In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score


# We might not need player_data.csv and players.csv since they are personal information that we can not make use of
salary_20 = pd.read_csv("salary_0020.csv")
salary_17 = pd.read_csv("salary_1718.csv")
stats = pd.read_csv("1950/seasons_stats.csv")
stats.drop(axis=1, index=0)
pd.set_option('display.max_columns', None)
print(stats.tail(5))

# Append salary to the dataframe
# Use the following attributes to fit models
# age, G, GS, MP, PER, TS, FTr, ORB%, DRB%, TRB%, AST%  STL%  BLK% TOV%  USG%, OWS  DWS, WS  WS/48, VORP, OBPM  DBPM  BPM and everything afterwards

       Unnamed: 0    Year             Player Pos   Age   Tm     G    GS  \
24686       24686  2017.0        Cody Zeller  PF  24.0  CHO  62.0  58.0   
24687       24687  2017.0       Tyler Zeller   C  27.0  BOS  51.0   5.0   
24688       24688  2017.0  Stephen Zimmerman   C  20.0  ORL  19.0   0.0   
24689       24689  2017.0        Paul Zipser  SF  22.0  CHI  44.0  18.0   
24690       24690  2017.0        Ivica Zubac   C  19.0  LAL  38.0  11.0   

           MP   PER    TS%   3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  \
24686  1725.0  16.7  0.604  0.002  0.442   8.6  17.3  12.9   9.1   1.8   3.0   
24687   525.0  13.0  0.508  0.006  0.247   9.2  17.0  13.2  12.2   0.7   3.3   
24688   108.0   7.3  0.346  0.000  0.161  10.8  24.9  17.6   5.3   0.9   3.7   
24689   843.0   6.9  0.503  0.448  0.181   1.9  14.2   8.0   6.1   0.9   1.5   
24690   609.0  17.0  0.547  0.013  0.206   7.1  21.9  14.3   8.1   1.1   4.4   

       TOV%  USG%  blanl  OWS  DWS   WS  WS/48  blank2  OBPM  DBPM  